In [46]:
import math
import pandas as pd
import numpy as np
import random
from sympy.ntheory.factor_ import totient
import matplotlib.pyplot as plt
from datetime import datetime
from math import ceil, sqrt
import time
import tracemalloc

# Die Programme sind in mehrere Zellen untergebracht. 
# Unter jedem Programm befinden sich seine Tests.

In [47]:
"""
Diese Zelle muss ausgefüllt werden, wenn das Ergebnis in Form einer latex Tabelle
mit allen Zwischenschritten für die Faktorisierung angezeigt werden soll.
"""
 
def to_Latex_Table(df):
    latex_table = df.reset_index().rename(columns={'index': '$i$',
                                                   'x_i rem N': '$x_i \mod N$',
                                                   'x_i rem p': '$x_i \mod p$',
                                                   'y_i rem N': '$y_i \mod N$',
                                                   'y_i rem p': '$y_i \mod p$',
                                                   'gcd(x_i − y_i, N)': '$gcd(x_i - y_i, N)$'
                                                    }).to_latex(
                                                    index=False,
                                                    column_format="|c||c||c||c||c||c|",escape=False).replace('\\\n', '\\ \hline\n').replace('\midrule','').replace('\\toprule','\hline').replace('\\bottomrule','')
    return latex_table
  
def to_Df_Table(df, x_i_rem_N, x_i_rem_p, y_i_rem_N, y_i_rem_p, gcd):
  new_row = {'x_i rem N': x_i_rem_N, 'x_i rem p': x_i_rem_p, 'y_i rem N': y_i_rem_N,'y_i rem p': y_i_rem_p ,'gcd(x_i − y_i, N)':gcd}
  df.loc[len(df)] = new_row
  return

  

In [26]:
#-------------------------------------------------------------------------
#-------------Pollards Rho-Methode zurFaktorisierung----------------------
#-------------------------------------------------------------------------
"""
Ausgabe: entweder ein Tupel mit den Ergebnissen (p, Laufzeit, Schritte)
         oder eine LaTeX-Tabelle mit allen Zwischenschritten.
Eingabe:        
Parameter n: Zu faktorisierende Zahl        
Parameter x_0: Startwert
Parameter table: Auf 1 setzen, wenn eine Tabelle mit allen Zwischenschritten
                       als Ausgabe gewünscht wird.
                Auf 0 setzen, wenn man Ergebnis-Tupel (p, Laufzeit, Schritte)
                       als Ausgabe sehen möchte.
Parameter p: ist ein Primfaktor von n. Falls es nicht gegeben ist muss auf 0 gesetzt werden
"""
def pollard_Rho_Factorization(n, x_0, table, p):
    x_i = x_0
    y_i = x_0
    st = time.time()

    gdc_Xi_Yi = 1 # math.gcd(abs(x_i - y_i),n)
    #print(gdc_Xi_Yi)
    if table == 1 and p >= 1:  
        dot_op = []
        op_table = pd.DataFrame(columns=['x_i rem N','x_i rem p', 'y_i rem N','y_i rem p', 'gcd(x_i − y_i, N)'])
        dot_op.append(x_0%p)
        to_Df_Table(op_table,x_i,x_i%p,y_i,y_i%p,gdc_Xi_Yi)


    steps= 0
   
    while gdc_Xi_Yi == 1:
        x_i = (pow(x_i,2) + 1)%n
        y_i = (pow(y_i,2) + 1)%n
        y_i = (pow(y_i,2) + 1)%n
        #gdc_Xi_Yi = egcd(abs(x_i - y_i),n)#
        gdc_Xi_Yi = math.gcd(abs(x_i - y_i),n)
        if gdc_Xi_Yi == n:
            return "failure"
        if table == 1 and p >= 1:
            to_Df_Table(op_table,x_i,x_i%p,y_i,y_i%p,gdc_Xi_Yi)
        steps = steps+1

    et = time.time()
    runTime = '{:.3f}'.format(et - st)
    if table == 1 and p >= 1:
        print(to_Latex_Table(op_table))    
        return 
        
    return (gdc_Xi_Yi, runTime, steps)

In [33]:
# ------------------------------------------------------
# Test-1 Faktorisierung  
# Aufgabe 3.23 (A sample run of Pollard’s rho) on Zur Gathen, CryptoSchool. 
# ------------------------------------------------------
# Die Anzahl der Iterationen in der Code muss angepasst werden, um die gleiche Anzahl an
# Iterationen wie im Buch gefordert, also es auf 13 festgelegt werden.
n = 100181
x_0 = 399
pollard_Rho_Factorization(n,x_0, 1,17)

\begin{tabular}{|c||c||c||c||c||c|}
\hline
 $i$ & $x_i \mod N$ & $x_i \mod p$ & $y_i \mod N$ & $y_i \mod p$ & $gcd(x_i - y_i, N)$ \\ \hline

   0 &          399 &            8 &          399 &            8 &                   1 \\ \hline
   1 &        59021 &           14 &        84891 &           10 &                   1 \\ \hline
   2 &        84891 &           10 &        95168 &            2 &                   1 \\ \hline
   3 &        61828 &           16 &        77478 &            9 &                   1 \\ \hline
   4 &        95168 &            2 &         5433 &           10 &                   1 \\ \hline
   5 &        84920 &            5 &        39918 &            2 &                   1 \\ \hline
   6 &        77478 &            9 &        91894 &            9 &                  17 \\ \hline

\end{tabular}



In [34]:
#------------------------------------------------------
# Test-2 Faktorisierung  Laufzeit, Iterationen vergleich
#------------------------------------------------------
#Liste der Eingabe.
semi_primes = [(874747,499,1753,20),
               (1000277423,15683,63781,30),
               (958132372843,518761,1846963,40),
               (755166234294083,12761317,59176199,50),
               (942010946063340029,2060746081,457121309,60),
               (851305247855690067257,16003938631,53193483647,70),
               (1034586769859873694285793,1999941224053,517308587581,80),
               (1001106354645776158228982593,58606506700639,17081829493087,90),
               (36362141541106933513583264081,137818528763917,137818528763917,95),
               (987097419540785456061948358783,542126391535789,1820788352960347,100),
               (38519733767557047415396818267083,4387479110584699,8779468299832817,105),
               (1218012219085603550747528231339059,17286713076734467,70459445568335377,110)
               ]

ex1 = pd.DataFrame(columns=['n = p*q','bit','Rechenzeit','$\sqrt p$', 'Iteration'])
for x in semi_primes:
    (semi_prim,p,q,bit) = x
    
    # Pollard auf alle Werte in der Liste anwenden
    (gdc_Xi_Yi,runTime,steps) = pollard_Rho_Factorization(semi_prim,2, 0, p)

    new_row = {'n = p*q': str(p)+'*'+str(q),
               'bit':bit,'Rechenzeit': runTime,
               '$\sqrt p$':int(math.sqrt(gdc_Xi_Yi)),
               'Iteration': steps}
    
    ex1.loc[len(ex1)] = new_row
    latex_table = ex1.to_latex(index=False,
                                column_format="|c||c||c||c||c||c|",
                                escape=False).replace('\\\n', '\\ \hline\n').replace('\midrule','').replace('\\toprule','\hline').replace('\\bottomrule','')
print(latex_table) 



\begin{tabular}{|c||c||c||c||c||c|}
\hline
                n = p*q & bit & Rechenzeit & $\sqrt p$ & Iteration \\ \hline

               499*1753 &  20 &      0.000 &        22 &        41 \\ \hline
            15683*63781 &  30 &      0.001 &       125 &        67 \\ \hline
         518761*1846963 &  40 &      0.006 &       720 &       216 \\ \hline
      12761317*59176199 &  50 &      0.046 &      3572 &      2530 \\ \hline
   2060746081*457121309 &  60 &      0.347 &     45395 &     19740 \\ \hline
16003938631*53193483647 &  70 &      1.467 &    126506 &     56224 \\ \hline

\end{tabular}



In [41]:
#-------------------------------------------------------
# Das erwähnte Beispiel im Test 2 Faktorisierung. (lange Laufzeit 8 min)
#-------------------------------------------------------
# n = p*q = 60643710950434723 ∗ 1517005143751371
p = 60643710950434723
n = 919968214479808246816905722987053
x_0 = 2
res = pollard_Rho_Factorization(n,2, 0, p)
print("(p,Rechenzeit,Iteration) = ", res)

(p,Rechenzeit,Iteration) =  (60643710950434723, '484.105', 111486076)


In [35]:
#--------------------------------------------------------------------
# -----------------(Baby-Step-Giant-Step-Algorithmus)----------------
#--------------------------------------------------------------------
#Ausgebe: (dlog_g, m, steps, runTime)
#Eingebe:
#        n: |G|
#        g: erzeuger
#        h: ein element in G     
# this is
def bs_gs(n, g, h):

   st = time.time()
   dlog_g = -1
   phi = totient(n)
   #m = ceil(math.isqrt(phi))
   m = ceil(sqrt(phi))
   r = 0
   c = 0

   # Erster baby-step
   bs = 1
   
   steps = 0
   ## dictionary für baby-steps {bs:key,..}
   bs_l = {}

   while r < m :
      # neue baby-step zu baby-steps dictionary hizufügen
      bs_l[bs] = r
      bs = pow(bs*g,1,n)
      r = r + 1
      
 
   
   # Erster giant-step
   g_pow_m = pow(g,-m,n)
   gs = h
   while c < m :
      # Überprüfe , ob gs im Baby-Steps-dictionary enthalten ist
      if gs in bs_l:
         steps = c+r
         r = bs_l[gs]
         # Berechnung des diskreten Logarithmus
         dlog_g = (c*m+r) % n
         et = time.time()
         runTime = '{:.3f}'.format(et - st)
         return (dlog_g, m, steps, runTime)
      
      # Falls giant-step nicht gefunden in baby-steps dictionary
      # berechne neue giant-step
      gs = gs*g_pow_m%n
      c = c + 1
   if dlog_g == -1:
      steps = c+r
      et = time.time()
      runTime = '{:.3f}'.format(et - st)
      return (0, m, steps,  runTime)
   
def verifyDlogSolution(n,g,h,a):
    return pow(g,a,n) == h  

In [36]:
# Hier kann man man BSGS Testen
# Die Ausgabe wird getestet auf ihre Korrektheit.
(n,g,h) = (7,3,2)
res  = bs_gs(n,g,h)
print("(n,g,h)             = ", (n,g,h))
print("dlog(h)             = ", res[0])
print("erwartete Schritte  = ", res[1])
print("benoetigte Schritte = ", res[2])
print("Laufzeit            = ", res[3])
print("Überprüfung         =", verifyDlogSolution(n,g,h,res[0]))

(n,g,h)             =  (7, 3, 2)
dlog(h)             =  2
erwartete Schritte  =  3
benoetigte Schritte =  3
Laufzeit            =  0.001
Überprüfung         = True


In [41]:
#--------------------------------------------------------------------
# ----------------------(Pollards Rho-Algorithmus)-------------------
#------------------(zur Lösung des diskreten Logarithmus)------------
#--------------------------------------------------------------------
#Ausgebe: (dlog_g, m, steps, runTime)
#Eingebe:
#        n: |G|
#        g: erzeuger
#        h: ein element in G


# Iteration Funktion 
# Aufteilung der Eingabegruppe mittels Modulo-Operator.

def update_x_u_v(n,z,g,h,u,v,phi):
    res = z%3
    if res == 2:
        z = (h*z)%n
        u = (u+1)%phi
    elif res == 0:
        z = (z*z)%n
        u = (2*u)%phi
        v = (2*v)%phi
    elif res == 1:
        z = (g*z)%n
        v = (v+1)%phi
    return (z,u,v)

# Iteration Funktion 
# Aufteilung der Eingabegruppe mittels Division.
"""
def update_x_u_v(n,z,g,h,u,v,phi):
    if  z < (n/3):
        z = (h*z)%n
        u = (u+1)%phi
    elif  z < (2*n)/3:
        z = (z*z)%n
        u = (2*u)%phi
        v = (2*v)%phi
    elif  z < n:
        z = (g*z)%n
        v = (v+1)%phi
    return (z,u,v)
"""
def pollard_dis(n, g, h):
    st = time.time()
    # Ordnung der Gruppe Berechnen
    phi = totient(n)
    
    expected_steps = ceil(math.isqrt(n))
    computation_type = None
    
     
    u_0 = (random.randint(0,int(phi/3)))
    v_0 = (random.randint(0,int(phi/3)))

    # Für Test 6 Nutzen sie dieser startwert
    #u_0 = 0 
    #v_0 = 0 

    u_y = u_0
    v_y = v_0
    u_x = u_0
    v_x = v_0
    
    x_0 = (pow(h,u_x)*pow(g,v_x))%n
    x = x_0
    y = x_0
    steps = 0
    w = 0
    while steps<10*expected_steps:
        steps = 1 + steps
        (x,u_x,v_x) = update_x_u_v(n,x,g,h,u_x,v_x,phi)
        (y,u_y,v_y) = update_x_u_v(n,y,g,h,u_y,v_y,phi)
        (y,u_y,v_y) = update_x_u_v(n,y,g,h,u_y,v_y,phi)
        if x == y:
            
            w = math.gcd((u_x-u_y),phi)
            break
    if w == 1:
        computation_type = "Variante 1"
        inverse_u = pow(u_x-u_y,-1,phi)
        v = v_y-v_x
        dlog_g = (inverse_u*v)%phi
        et = time.time()
        runTime = '{:.3f}'.format(et - st) 
        return (dlog_g ,computation_type, (x_0,u_0,v_0) ,(x,u_x,v_x) ,(y,u_y,v_y), steps, expected_steps,runTime)
    elif w > 1 :
        computation_type = "Variante 2"
        u = u_x-u_y
        v = v_y-v_x
        inverse_u_div_w = pow(int((u)/w),-1,int(phi/w))
        v_div_w = int((v)/w)
        phi_w = int(phi/w)
        b = (inverse_u_div_w*v_div_w)%phi_w
        j=0
        while j < w:
            dlog_g = (b+j*phi_w)%phi
            g_power_a = pow(g,dlog_g,n)
            j = j + 1 
            if g_power_a == h:
                et = time.time()
                runTime = '{:.3f}'.format(et - st)         
                return (dlog_g ,computation_type, (x_0,u_0,v_0) ,(x,u_x,v_x) ,(y,u_y,v_y), steps, expected_steps,runTime)
            elif g_power_a != h and j == w-1:
                print("failure: Der diskrete Logarithmus kann nicht berechnet werden")
                et = time.time()
                runTime = '{:.3f}'.format(et - st)
                return (-1 , 0, 0, 0, 0, steps, expected_steps,runTime)
             
    else:
        et = time.time()
        runTime = '{:.3f}'.format(et - st)
        print("failure: Der diskrete Logarithmus kann nicht berechnet werden")
        return (dlog_g , 0, 0, 0, 0, steps, expected_steps,runTime)
    
def verifyDlogSolution(n,g,h,a):
    return pow(g,a,n) == h 

In [42]:
# ------------------------------------------------------
# Test-3 Seite 22, 23
# Diskrete Logarithmus
# Aufgabe 4.12 (Pollard rho method) on Zur Gathen, CryptoSchool. 
# ------------------------------------------------------
(n,g,h) = (1000003,2,43116)
res = pollard_dis(n,g,h)
#print(res)
print("(n,g,h)             = ", (n,g,h))
print("dlog(h)             = ", res[0])
print("Berechnungsart      = ", res[1])
print("(x_0,u_0,v_0)       = ", res[2])
print("(x,u_x,v_x)         = ", res[3])
print("(y,u_y,v_y)         = ", res[4])
print("erwartete Schritte  = ", res[6])
print("benoetigte Schritte = ", res[5])
print("Laufzeit            = ", res[7])
print("Überprüfung         = ", verifyDlogSolution(n,g,h,res[0]))


(n,g,h)             =  (1000003, 2, 43116)
dlog(h)             =  276162
Berechnungsart      =  Variante 2
(x_0,u_0,v_0)       =  (292262, 116066, 121443)
(x,u_x,v_x)         =  (877280, 678413, 741287)
(y,u_y,v_y)         =  (877280, 27149, 750347)
erwartete Schritte  =  1000
benoetigte Schritte =  4000
Laufzeit            =  0.252
Überprüfung         =  True


In [ ]:
# -------------------------------------------------------------
# Test-4, Test-5 Seite 23, 24, 25
# Diskrete Logarithmus
# Für Test 4: verwenden Sie tries = 100
# Für Test 5: verwenden Sie tries = 100 oder 1000, und Und die Methode
#             zur Gruppenaufteilung mithilfe des Modulo-Operators in der
#             darüberliegenden Zelle .
# --------------------------------------------------------------
test2_df = pd.DataFrame(columns=['dlog(h)','Berechnungsart','$(x_0,u_0,v_0)$',
                              '$(x_i,v_i,u_i)$','$(y_j,v_j,u_j)$','benoetigte Schritte',
                              'erwartete Schritte','Laufzeit'])

for i in range(10):
    res = pollard_dis(1000003,2,43116)
    new_row = {'dlog(h)': res[0],'Berechnungsart': res[1],'$(x_0,u_0,v_0)$': res[2],
                              '$(x_i,v_i,u_i)$': res[3],'$(y_j,v_j,u_j)$': res[4],
                              'benoetigte Schritte': res[5], 'erwartete Schritte': res[6],
                              'Laufzeit': res[7]}
    test2_df.loc[len(test2_df)] = new_row
    

# Latextabelle des Ergebnisses
test2_latex_table = test2_df.to_latex(index=True,column_format="|c||c||c||c||c||c||c||c||c|",escape=False).replace('\\\n','\\ \hline\n').replace('\midrule','').replace('\\toprule','\hline').replace('\\bottomrule','')
print(test2_latex_table)
 

In [ ]:
# -------------------------------------------------------------
# Test-6, Seite 25
# Pollards Rho-Algorithmus Teil
# Diskrete Logarithmus 
# Bevor Sie diesen Test ausführen, sollte (u_0,v_0) auf (0,0) gesetzt werden.
# Verwenden Sie die Methode zur Gruppenaufteilung mithilfe des Modulo-Operators in der
# Pollards Rho-Algorithmus-Program Zelle 
# Wegen des letzten Testfalls ist darauf zu achten,
# dass dieser Test länger als 15 Minuten dauert.
# --------------------------------------------------------------

# Test-4 DLP Pollard-Teil
# 
DlP_test4_List_Pollard_BSGS = [(7,3,2),(67,2,17),(809,2,201),
                                (5939,3,992),(69941,2,11331),
                                (681179,3,10031),(5281751,3,13413),
                                (51936127,2,100113),(937521743,2,1000113),
                                (4788348977,2,3241), (18593095387,3,11157295687),
                                (899765412709,3,215295687),(8783848257259,7,15295687),
                                (85569430093193,3,331225295687),(363686635634083, 2, 122225295687),
                                (6162957457875851,3,111225295687)]

test4_DLP_df = pd.DataFrame(columns=['Test','$(x_0,u_0,v_0)$','dlog(h)', 'erwartete Schritte', 'benoetigte Schritte', 'Laufzeit', 'Speicherbedarf'])

for test in DlP_test4_List_Pollard_BSGS:

    tracemalloc.start()
    tracemalloc.clear_traces()
    tracemalloc.reset_peak()

    res = pollard_dis(test[0],test[1],test[2])

    memory = '{:.3f}'.format(tracemalloc.get_traced_memory()[1]/(1024*1024))
    tracemalloc.stop()

    new_row = {'Test': test,'$(x_0,u_0,v_0)$': res[2],'dlog(h)': res[0],
               'erwartete Schritte': res[6], 'benoetigte Schritte': res[5],
                 'Laufzeit': res[7],'Speicherbedarf':memory}
    test4_DLP_df.loc[len(test4_DLP_df)] = new_row
test4_DLP_latex_table = test4_DLP_df.to_latex(index=True,column_format="|c||c||c||c||c||c||c||c||c|",escape=False).replace('\\\n','\\ \hline\n').replace('\midrule','').replace('\\toprule','\hline').replace('\\bottomrule','')
print(test4_DLP_latex_table)

In [40]:
# -------------------------------------------------------------
# Test-6, Seite 25
# Pollards BS-GS-Algorithmus Teil
# Diskrete Logarithmus 
# Bevor Sie diesen Test ausführen, sollte (u_0,v_0) auf (0,0) gesetzt werden.
# Letzter Testfall wurde aufgrund des hohen Speicherbedarfs ausgeschlossen.
# Achtung: dieser Test länger als 10 Minuten dauert
# --------------------------------------------------------------
DlP_test4_List_Pollard_BSGS = [(7,3,2),(67,2,17),(809,2,201),
                                (5939,3,992),(69941,2,11331),
                                (681179,3,10031),(5281751,3,13413),
                                (51936127,2,100113),(937521743,2,1000113),
                                (4788348977,2,3241), (18593095387,3,11157295687),
                                (899765412709,3,215295687),(8783848257259,7,15295687),
                                (85569430093193,3,331225295687),(363686635634083, 2, 122225295687)]#,
                                #(6162957457875851,3,111225295687)]

test4_DLP_df_BSGS = pd.DataFrame(columns=['Test','dlog(h)', 'erwartete Schritte', 'benoetigte Schritte', 'Laufzeit','Speicherbedarf'])

for test in DlP_test4_List_Pollard_BSGS:
    tracemalloc.start()
    tracemalloc.clear_traces()
    tracemalloc.reset_peak()
    res = bs_gs(test[0],test[1],test[2])
    memory = '{:.3f}'.format(tracemalloc.get_traced_memory()[1]/(1024*1024))

    new_row = {'Test': test, 'dlog(h)': res[0],'erwartete Schritte': res[1], 'benoetigte Schritte': res[2], 'Laufzeit': res[3],'Speicherbedarf':memory}
    test4_DLP_df_BSGS.loc[len(test4_DLP_df_BSGS)] = new_row
test4_DLP_BSGS_latex_table = test4_DLP_df_BSGS.to_latex(index=True,column_format="|c||c||c||c||c||c||c||c||c|",escape=False).replace('\\\n','\\ \hline\n').replace('\midrule','').replace('\\toprule','\hline').replace('\\bottomrule','')
print(test4_DLP_BSGS_latex_table)

\begin{tabular}{|c||c||c||c||c||c||c||c||c|}
\hline
{} &               Test & dlog(h) & erwartete Schritte & benoetigte Schritte & Laufzeit & Speicherbedarf \\ \hline

0 &          (7, 3, 2) &       2 &                  3 &                   3 &    0.000 &          0.000 \\ \hline
1 &        (67, 2, 17) &      64 &                  9 &                  16 &    0.000 &          0.000 \\ \hline
2 &      (809, 2, 201) &     117 &                 29 &                  33 &    0.000 &          0.002 \\ \hline
3 &     (5939, 3, 992) &    1427 &                 78 &                  96 &    0.000 &          0.004 \\ \hline
4 &  (69941, 2, 11331) &    8258 &                265 &                 296 &    0.001 &          0.018 \\ \hline

\end{tabular}

